In [159]:
base_address = None
limit = None
access_rights = None

class SegmemtType:
    CODE = 0
    DATA = 1

def make(base_address: bytes, limit: bytes, /, segment_type: SegmemtType = SegmemtType.CODE, readable = False, executable = False, writable = False) -> bytes:
    descriptor = bytearray()
    
    if len(base_address) < 4:
        raise ValueError("base_address must be 4 bytes")
    
    if len(limit) < 4:
        raise ValueError("limit must be 4 bytes")
    
    base_31_24 = base_address[0]
    base_23_16 = base_address[1]
    base_15_00 = base_address[2:]
    
    limit_19_16 = limit[2]
    limit_15_00 = limit[2:]
    
    limit_19_16 = limit_19_16 >> 4
    limit_19_16 = limit_19_16 << 4
    limit_19_16 = limit[2] - limit_19_16
    
    descriptor.extend(base_31_24.to_bytes(length=1, byteorder='big'))
    
    g_d_avl = 0xF << 4
    g_d_avl = g_d_avl + limit_19_16
    
    descriptor.extend(g_d_avl.to_bytes(length=1, byteorder='big'))
    
    p_dpl_s = 0xE << 4
    
    d_type = 0x0000
    
    if readable:
        d_type = d_type | 0b10
    
    if executable or writable:
        d_type = d_type | 0b100
    
    if segment_type == SegmemtType.CODE:
        d_type = d_type | 0b1000
    
    p_dpl_s = p_dpl_s + d_type
    
    descriptor.extend(p_dpl_s.to_bytes(length=1, byteorder='big'))
    
    descriptor.extend(base_23_16.to_bytes(length=1, byteorder='big'))
    descriptor.extend(base_15_00)
    descriptor.extend(limit_15_00)
    
    return bytes(descriptor)

def describe(descriptor: bytes) -> None:
    descriptor = bytearray(descriptor)
    
    print("Descriptor bytes:\n")
    for b in descriptor:
        print(f"{descriptor.index(b)} => {hex(b)}")
        
    print(f"\n=== Descriptor parts ===\n")
    
    print(f"=== Base address ===")
    base_31_24 = descriptor[0]
    base_23_00 = descriptor[::-1][2:5][::-1]
    full_base_address = bytearray()
    full_base_address.extend(base_31_24.to_bytes(length=1, byteorder='big'))
    full_base_address.extend(base_23_00)
    full_base_address_str = ''.join(format(x, '02x') for x in full_base_address)
    print(f"Full base address: {full_base_address_str}")
    
    print(f"\n=== Segment limit ===")
    
    limit_15_00 = descriptor[::-1][:2][::-1]
    limit_19_16 = descriptor[1]
    limit_19_16 = limit_19_16 >> 4
    limit_19_16 = limit_19_16 << 4
    limit_19_16 = descriptor[1] - limit_19_16
    
    full_segment_limit = bytearray()
    full_segment_limit.extend(limit_19_16.to_bytes(length=1, byteorder='big'))
    full_segment_limit.extend(limit_15_00)
    
    full_segment_limit_str = ''.join(format(x, '02x') for x in full_segment_limit)
    print(f"Full segment limit: {full_segment_limit_str}")
    
    print(f"\n=== Special bits ===")
    
    print("G | D | 0 | AVL")
    print("-" * 15)
    
    g_d_avl = descriptor[1]
    g_d_avl = g_d_avl >> 4
    print(f"{(g_d_avl & 0b1000) >> 3} | {(g_d_avl & 0b0100) >> 2} | {(g_d_avl & 0b0010) >> 1} | {g_d_avl & 0b0001}\n")
    
    print("P | DPL | S")
    print("-" * 12)
    
    p_dpl_s = descriptor[2] >> 4
    dpl = (p_dpl_s >> 1) & 0b11
    print(f"{(p_dpl_s & 0b1000) >> 3} | {bin(dpl)[2:]}  | {p_dpl_s & 0b0001}\n")
    
    seg_type = descriptor[2] >> 4
    seg_type = seg_type << 4
    seg_type = descriptor[2] - seg_type
    seg_type_desc =  seg_type
    
    seg_type_desc = seg_type_desc >> 3
    seg_bit = 0
    
    if seg_type_desc:
        print("=== Code segment ===")
        print("1 | C | R | A")
        seg_bit = 1
    else:
        print("=== Data segment ===")
        print("0 | E | W | A")
        seg_bit = 0
    print("-" * 15)
    print(f"{seg_bit} | {(seg_type & 0b0100) >> 2} | {(seg_type & 0b0010) >> 1} | {seg_type & 0b0001}")
    

In [160]:
import random

describe(make(random.randbytes(4),random.randbytes(4), SegmemtType.DATA, True, True))

Descriptor bytes:

0 => 0x11
1 => 0xfe
2 => 0xe6
3 => 0xed
4 => 0x7b
5 => 0xb5
6 => 0xce
7 => 0x9f

=== Descriptor parts ===

=== Base address ===
Full base address: 11ed7bb5

=== Segment limit ===
Full segment limit: 0ece9f

=== Special bits ===
G | D | 0 | AVL
---------------
1 | 1 | 1 | 1

P | DPL | S
------------
1 | 11  | 0

=== Data segment ===
0 | E | W | A
---------------
0 | 1 | 1 | 0
